<center>

## ENSE 350 – Math Programming for Software Engineers - Laboratory

# Lab 3: RSA Cryptosystem

### University of Regina
### Faculty of Engineering and Applied Science - Software Systems Engineering
### Lab Instructor: [Adam Tilson](mailto:Adam.Tilson@uregina.ca)

</center>

### Part 1: Object-Oriented Programming Review

In [6]:
import math

# Utility function
def get_distance (coord_a, coord_b):
    """Find the distance between two cartesian coordinates
    >>> get_distance ((0,1), (0,0))
    1.0
    """
    return math.sqrt((coord_a[0] - coord_b[0]) ** 2 + (coord_a[1] - coord_b[1]) ** 2)

class RightTriangle:
    """This class represents a right triangle
    Add the following doctests...
    
    Creating the triangle with vertices: (0,0), (0,3) and (4,0)
    >>> right_triangle = RightTriangle((0,0), (0,3), (4,0))
    
    Getting the hypotenuse for the for the previous triangle
    >>> right_triangle.get_hypotenuse()
    5.0
    
    Getting the perimeter
    >>> right_triangle.get_perimeter()
    12.0
    
    Creating a failed triangle with coordinates (0,0), (1,2) and (2,1) should raise a value error
    >>> fail_triangle = RightTriangle((0,0), (1,2), (2,1))
    Traceback (most recent call last):
        ...
    ValueError: Not a right triangle

    """
    
    # __init__ is the constructor function
    def __init__ (self, coord_a, coord_b, coord_c):
        
        # check if the coordinates consitute a right triangle, otherwise throw an error
        
        self.coord_a = coord_a
        self.coord_b = coord_b
        self.coord_c = coord_c
        
        self.side_lengths = []
        self.side_lengths.append (get_distance (coord_a, coord_b))
        self.side_lengths.append (get_distance (coord_a, coord_c))
        self.side_lengths.append (get_distance (coord_b, coord_c))
        
        self.side_lengths.sort()
        
        if self.side_lengths[0] **2 + self.side_lengths[1] ** 2 - self.side_lengths[2] ** 2 > 0.0001:
            raise ValueError("Not a right triangle")
    
    def get_hypotenuse (self):
        return self.side_lengths[-1]
    
    def get_perimeter (self):
        perimeter = 0
        for side_length in self.side_lengths:
            perimeter += side_length
        return perimeter
        
import doctest
doctest.testmod()

TestResults(failed=0, attempted=5)

### Part 2: RSA Cryptosystem Overview

In this example we will implement a simple "Right Triangle" class to demonstrate how object oriented programming works in python.

The following communications diagram describes the architecture we will use in the lab assignment. The order of operations which would need to occur are labeled on the arrows. The arrows show which objects are calling functions on which other objects.

![](res/comms.png)

### Part 3: RSA Cryptosystem Setup

- The generator creates a public key and a private key as follows
  1. Generate two distinct primes, $p$, $q$. These must be kept secret.
  2. Let $n = pq, \phi(n) = (p-1)(q-1)$
  3. Select an integer $e$ such that $\text{gcd}(e, \phi(n)) = 1$
    - The public key pair $(e,n)$ should be widely distributed
  4. Compute $d$ such that $de \equiv 1 (\text{mod } \phi(n))$
    - This can be done using the pulverizer
    - The private key pair $(d,n)$ must be kept secret!

### Part 4: RSA Crytposystem Encryption / Decryption

Encoding:

- Given a message, $m$, the sender first checks that $\text{gcd}(m,n) = 1$
- The sender encrypts message $m$ to produce $m'$ using the public key:

  - $m' = \text{rem }(m^e, n))$
  - $m^e \equiv m' (\text{mod } n)$
  
Decoding:

- The receiver decrypts message, $m'$ back to message $m$ using the secret key:
  - $m = \text{rem}((m')^d,n)$
  - $(m')^d \equiv m (\text{mod }n)$
  
Because taking large powers is computationally intensive, we instead use `Repeated Squares`

### Part 5: Full Example

1 Initial Setup:
- Let $p = 7$
- Let $q = 19$
- $n = p \cdot q = 7 \cdot 19 = 133$
- $\phi(n) = (7-1) \cdot (19-1) = 6 \cdot 18 = 108$

2 Public Key Generation:
- Let $e = 17$, (given. $e$ must be relatively prime to $\phi(n)$)
- public key = $(17, 133)$

3 Private Key Generation (Pulverizer)
- $e \cdot d \equiv e \cdot d \equiv 1 (\text{mod }\phi(n))$
- $k \cdot \phi(n) + e \cdot d \equiv 1 (\text{mod }\phi(n)), k \in \mathbb{Z}$
- $k \cdot \phi(n) + d \cdot 17 \equiv  1 (\text{mod }\phi(n))$

This is similar to the outcome of the pulverizer:
- $s \cdot a + t \cdot b \equiv 1 (\text{mod } \phi(n))$

Where:
- $s = k$
- $a = \phi(n) = 108$, 
- $d=t$
- $e=17$
* We are looking for $t$. We don't really care about $s$

- Pulverizer $(108, 17)$

|$x$|$y$|$q$|$r$|$r = x-q \cdot y$|$s$|$t$|
|---|---|---|---|---|---|---|
|$108$|$17$|$6$|$6$ |$6 = 108 - 6 \cdot 17$|$1$|$-6$|
|$17$|$6$|$2$|$5$ |$5 = 17 - 2 \cdot 6$|||
|||||$5 = -2 \cdot 108 + 13 \cdot 17$|$-2$|$13$|
|$6$|$5$|$1$|$1$|$1 = 6 - 1 \cdot 5$|||
|||||$1 = 3 \cdot 108 - 19 \cdot 17$|$3$|$-19$|

$1 = 3 \cdot 108 - 19 \cdot 17$

$-19$ is our value for $d$. inverse. But we want our inverse to be positive!

Since we don't care about $s$, can we add or subtract some value to make it positive, taking it out of s? 

- $1 = 3 \cdot 108 - 19 \cdot 17$

- $1 = sa + tb$
- $1 = (sa + tb) + k(ba - ab), k \in \mathbb{Z}, k \geq = 1$

- $1 = 3 \cdot 108   - 19 \cdot 17 + 1 \cdot (-17 \cdot 108  + 108 \cdot 17)$
- $1 = -14 \cdot 108 + 89 \cdot 17$

Note, we can adjust k until we have a positive value

$89$ is our positive inverse, the private key is $d=(89, 133)$

4 Encryption, m=`h`$=104$
- $m'=104^{17} (\text{mod }133)$
- Use repeated squaring...
- $104 \equiv 104 (\text{mod }133)$
- $104^2 \equiv 43 (\text{mod }133)$
- $104^4 \equiv 120 (\text{mod }133)$
- $104^8 \equiv 36 (\text{mod }133)$
- $104^{16} \equiv 99 (\text{mod }133)$
- $104^{17} = 104 \cdot 104^{16} \equiv 104 \cdot 99 (\text{mod }133) \equiv 55 (\text{mod }133)$
- Our encrypted message is $m' = 55$

5 Decryption, $m'=55$
- $m=55^{89} (\text{mod }133)$
- Use repeated squaring...
- $55 \equiv 55 (\text{mod }133)$
- $55^{2} \equiv 99 (\text{mod }133)$
- $55^{4} \equiv 92 (\text{mod }133)$
- $55^{8} \equiv 85 (\text{mod }133)$
- $55^{16} \equiv 43 (\text{mod }133)$
- $55^{32} \equiv 120 (\text{mod }133)$
- $55^{64} \equiv 36 (\text{mod }133)$

$55^{89} = 55^{64}\cdot 55^{16}\cdot 55^{8} \cdot 55^{1} \equiv 55 \cdot 85 \cdot 43 \cdot 36 (\text{mod } 133) \equiv 104 (\text{mod } 133)$

See also the included spreadsheet for more info on repeated squaring.